# Store data

As calling API everytime may takes a while and we may hit the limit of API, we will store the data from API for now. 

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [3]:
import imdb as ib
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile
import types
import numpy as np

In [4]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

# IMDB

In [6]:
imdb = IMDb()
#get a movie by id
imdb_movie = imdb.get_movie('0325980')
#access attributes of the movie by dictionary keys
imdb.update(imdb_movie)
#available keys
imdb_info_column  =imdb_movie.keys()
imdb_info_column  = [str(c) for c in imdb_info_column]
imdb_info_column.sort()

In [7]:
print(len(imdb_info_column))
print(imdb_info_column)

61
['akas', 'animation department', 'art department', 'art direction', 'aspect ratio', 'assistant director', 'camera and electrical department', 'canonical title', 'cast', 'casting department', 'casting director', 'certificates', 'cinematographer', 'color info', 'costume department', 'costume designer', 'countries', 'country codes', 'cover url', 'director', 'distributors', 'editor', 'editorial department', 'full-size cover url', 'genres', 'kind', 'language codes', 'languages', 'location management', 'long imdb canonical title', 'long imdb title', 'make up', 'miscellaneous companies', 'miscellaneous crew', 'mpaa', 'music department', 'original music', 'plot', 'plot outline', 'producer', 'production companies', 'production design', 'production manager', 'rating', 'runtimes', 'set decoration', 'smart canonical title', 'smart long imdb canonical title', 'sound crew', 'sound mix', 'special effects companies', 'special effects department', 'stunt performer', 'thanks', 'title', 'top 250 rank'

In [8]:
# IMDB API shows that we can retrieve information from proprty of movies object
imdb_info_column = imdb_movie.keys_alias.keys() 
imdb_info_column.sort()
print(len(imdb_info_column))
print(imdb_info_column)

85
['actors', 'actresses', 'aka', 'also known as', 'amazon review', 'art direction by', 'casting', 'casting by', 'certificate', 'certification', 'certifications', 'cinematography', 'cinematography by', 'color', 'costume and wardrobe department', 'costume design', 'costume design by', 'country', 'cover', 'created by', 'crew members', 'crewmembers', 'directed by', 'distribution', 'distribution companies', 'distribution company', 'distributor', 'editing', 'episodes cast', 'episodes number', 'faq', 'film editing', 'film editing by', 'frequently asked questions', 'full-size cover', 'genre', 'guest', 'guest appearances', 'lang', 'language', 'make-up', 'makeup', 'makeup department', 'merchandise', 'merchandising', 'misc companies', 'misc company', 'misc crew', 'miscellaneous', 'miscellaneous company', 'miscellaneous links', 'miscellaneouscrew', 'music', 'non-original music by', 'notable tv guest appearances', 'original music by', 'other companies', 'other company', 'other crew', 'parental gui

In [11]:
# IMDB API shows that we can retrieve information from proprty of movies object
imdb_info_column2 = imdb_movie.keys_alias.values() 
imdb_info_column2 = list(set(imdb_info_column2))
imdb_info_column2.sort()
print(len(imdb_info_column2))
print(imdb_info_column2)

49
['airing', 'akas', 'amazon reviews', 'art direction', 'assistant director', 'cast', 'casting director', 'certificates', 'cinematographer', 'color info', 'costume department', 'costume designer', 'countries', 'cover url', 'creator', 'director', 'distributors', 'editor', 'faqs', 'full-size cover url', 'genres', 'guests', 'languages', 'make up', 'merchandising links', 'misc links', 'miscellaneous companies', 'miscellaneous crew', 'non-original music', 'number of episodes', 'number of seasons', 'original music', 'parents guide', 'photo sites', 'plot', 'producer', 'production companies', 'production manager', 'rating', 'runtimes', 'set decoration', 'sound clips', 'sound crew', 'special effects', 'special effects companies', 'stunt performer', 'video clips', 'visual effects', 'writer']


In [15]:
# get IMDB movies that we already have data for TMDB
imdb_ids = tmdb_movies['imdb_id'].tolist()
tmdb_ids = tmdb_movies['id'].tolist()

imdb_ids = [ str(imdb_id.replace('tt','')) for imdb_id in imdb_ids]
imdb_ids = [ imdb_id for imdb_id in imdb_ids if imdb_id !='' and imdb_id is not None]

imdb_ids = list(set(imdb_ids))

In [115]:
imdb_info_column = imdb_movie.keys_alias.values()
imdb_info_column = list(set(imdb_info_column))
if 'imdb_id' not in imdb_info_column:
    imdb_info_column.append('imdb_id')
imdb_info_column.sort()

movies = pd.DataFrame(columns=imdb_info_column)
invalid_imdb_ids = []

imdb_filename = str(dir_data)+'\\drv_imdb_movie_info.json'
imdb_filename_backup = str(dir_data)+'\\drv_imdb_movie_info_bkp.json'
imdb_invalid_id_filename = str(dir_data)+'\\drv_imdb_movie_invalid_id.json'
count = 0
# if we already have a movie data file, we can just continue appending it instead of starting from id 1 again.
if os.path.isfile(imdb_filename):
    movies = pd.read_json(imdb_filename)
    if len(movies['imdb_id'].tolist()) >0:
        movie_ids = movies['imdb_id'].tolist()
        imdb_ids = [x for x in imdb_ids if x not in movie_ids]
        imdb_info_column = movies.columns
    else:
        movies = pd.DataFrame(columns=imdb_info_column)

# we don't want to waste our effort to load invalid imdb_id
if os.path.isfile(imdb_invalid_id_filename):
    invalid_imdb_id_df = pd.read_json(imdb_invalid_id_filename)
    if len(invalid_imdb_id_df['invalid_imdb_id'].tolist()) >0:
        invalid_imdb_ids = invalid_imdb_id_df['invalid_imdb_id'].tolist()
        imdb_ids = [x for x in imdb_ids if x not in invalid_imdb_ids]
        
for i in imdb_ids:
    count += 1
    if (count % 500 == 0):
        movies.to_json(path_or_buf= imdb_filename)
        if (len(invalid_imdb_ids)>0):
            invalid_imdb_id_df = pd.DataFrame({'invalid_imdb_id': invalid_imdb_ids}) 
            invalid_imdb_id_df.to_json(path_or_buf= imdb_invalid_id_filename)
    #skip the non-existing movie ids
    #print(i)
    try:
        imdb_movie = imdb.get_movie(i)
        imdb.update(imdb_movie)
        if (len(imdb_movie.keys()) > 0) :       
            movie_details = []
            for c in imdb_info_column:
                if imdb_movie.has_key(c):
                    info_field = imdb_movie[c]
                    if info_field is not None:
                        if type(info_field) is list:                    
                            if isinstance(info_field[0], ib.Person.Person):
                                info_list = []
                                for item in info_field:
                                    info_list.append(item.getID())
                                movie_details.append(info_list)
                            elif isinstance(info_field[0], ib.Company.Company):
                                info_list = []
                                for item in info_field:
                                    info_list.append(item.getID())
                                movie_details.append(info_list)  
                            else:
                                movie_details.append(info_field)
                        else:
                            movie_details.append(info_field)
                    else:
                        if c == "imdb_id":
                            movie_details.append(i)
                        else:
                            movie_details.append(None)      

                else:
                        movie_details.append(None)
                
            movies.loc[len(movies.index)] = movie_details
        
            movies.head(5)
    except Exception:
        invalid_imdb_ids.append(i)
        continue
        
movies.to_json(path_or_buf= imdb_filename)
if (len(invalid_imdb_ids)>0):
    invalid_imdb_id_df = pd.DataFrame({'invalid_imdb_id': invalid_imdb_ids}) 
    invalid_imdb_id_df.to_json(path_or_buf= imdb_invalid_id_filename)

2017-04-05 12:31:48,582 CRITICAL [imdbpy] C:\Users\Chrystal\Anaconda2\envs\py27\lib\site-packages\imdb\_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/title/tt0072996/combined', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', error(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')), 'errmsg': '[Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\Chrystal\Anaconda2\envs\py27\lib\site-packages\imdb\parser\http\__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "C:\Users\Chrystal\Anaconda2\envs\py27\lib

KeyboardInterrupt: 

In [116]:
#make a backup in case of corruption
copyfile(imdb_filename, imdb_filename_backup) 

In [117]:
imdb_filename = str(dir_data)+'\\drv_imdb_movie_info.json'
imdb_movies = pd.read_json(imdb_filename)
imdb_movies.head(5)

,airing,akas,amazon reviews,art direction,assistant director,cast,casting director,certificates,cinematographer,color info,...,runtimes,set decoration,sound clips,sound crew,special effects,special effects companies,stunt performer,video clips,visual effects,writer
0,NaN,[Ultimate Avengers 2: Rise of the Panther::USA...,NaN,None,None,"[1225431, 0217221, 0557219, 0941404, 0001882, ...",[0800493],"[Argentina:Atp::(DVD rating), Russia:12+, Russ...",None,[Color],...,[73],None,NaN,"[0039305, 1409862, 1886186, 1423589, 0190961, ...",NaN,None,None,NaN,[0777295],"[2092839, 1628806, 1080564, 1219736, 1080564, ..."
1,NaN,"[Bajo la piel::Argentina (imdb display title),...",NaN,None,"[0116502, 0348808, 0601635, 0931271]","[0164809, 0891895, 0252961, 0862328, 0001026, ...","[0058089, 0338418]","[Argentina:18, Australia:MA15+, Netherlands:12...",[0312790],[Color],...,[103],[0048585],NaN,"[2657542, 1434212, 1365757, 2850266, 0226886, ...",NaN,None,"[2603844, 0915676]",NaN,None,[0000586]
10,NaN,[Ultimate Avengers 2: Rise of the Panther::USA...,NaN,None,None,"[1225431, 0217221, 0557219, 0941404, 0001882, ...",[0800493],"[Argentina:Atp::(DVD rating), Russia:12+, Russ...",None,[Color],...,[73],None,NaN,"[0039305, 1409862, 1886186, 1423589, 0190961, ...",NaN,None,None,NaN,[0777295],"[2092839, 1628806, 1080564, 1219736, 1080564, ..."
100,NaN,"[One Wild Night::, USA (working title), Возмож...",NaN,[0055618],"[0742835, 0815340]","[0001844, 0000124, 0000551, 0612487, 0413698, ...",[0228938],"[Australia:PG, Germany:6, Iceland:L, South Kor...",[0005791],[Color],...,[83],[0460018],NaN,"[0018482, 0081662, 0003263, 0463296, 0872150]",NaN,None,"[0197780, 0258351, 2268592]",NaN,None,[0000455]
1000,NaN,"[Wicked City::Australia (imdb display title), ...",NaN,[0644470],None,"[0946344, 0810987, 0589645, 0297847, 0448950, ...","[0532146, 0810987, 3108562]","[Australia:R18+, Canada:18A::(Alberta/British ...",[0411060],[Color],...,[82],None,NaN,"[0262171, 1187913, 0593829, 0909603]",NaN,None,None,NaN,None,"[0159109, 0452837, 0593377, 0810987]"
